In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
import tensorflow as tf
import pickle
from tensorflow import keras
from tensorflow.keras import layers, metrics
from tensorflow.keras.metrics import Accuracy
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn import model_selection

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#DATA PREPARATION
raw_datas = pd.read_csv("/content/drive/MyDrive/Dataset TA/PIR dataset Time train 2 hari.csv")
number_of_day = raw_datas.shape[0]/86400
print("Total data = " + str(raw_datas.shape[0]) + " detik")
print("Total hari dalam data = " + str(int(number_of_day)) + " hari")

Total data = 172800 detik
Total hari dalam data = 2 hari


In [ ]:
#EXTRACTING DATA

datas = pd.read_csv("/content/drive/MyDrive/Dataset TA/PIR dataset Duration train 2 hari.csv")
file = open("/content/drive/MyDrive/Dataset TA/PIR last 30 list dataset train 2 hari.csv","w")
for z in range(30):
  file.write("Durasi")
  file.write(str(z+1))
  for y in range(4):
    file.write(",Data")
    file.write(str(z+1))
    file.write("|")
    file.write("Pir")
    file.write(str(y+1))
  file.write(",")
file.write("Gerakan")
file.write("\n")

percent1 = round(datas.shape[0]/100)
percentage = 0

for i in range(datas.shape[0]):
  if(i%percent1 == 0):
    print(str(percentage)+"%")
    percentage+=1
  if(i < 29):
    for u in range(29-i):
      file.write("0,0,0,0,0,")
    for e in range(i+1):
      for t in range(5):
        file.write(str(datas.iloc[e][t]))
        file.write(",") 
  else:
    for a in range(30):
      for o in range(5):
        file.write(str(datas.iloc[i-(29-a)][o]))
        file.write(",")
  file.write(datas.iloc[i][5])
  file.write("\n")

print("100%")
file.close()

0%
1%
2%
3%
4%
5%
6%
7%
8%
9%
10%
11%
12%
13%
14%
15%
16%
17%
18%
19%
20%
21%
22%
23%
24%
25%
26%
27%
28%
29%
30%
31%
32%
33%
34%
35%
36%
37%
38%
39%
40%
41%
42%
43%
44%
45%
46%
47%
48%
49%
50%
51%
52%
53%
54%
55%
56%
57%
58%
59%
60%
61%
62%
63%
64%
65%
66%
67%
68%
69%
70%
71%
72%
73%
74%
75%
76%
77%
78%
79%
80%
81%
82%
83%
84%
85%
86%
87%
88%
89%
90%
91%
92%
93%
94%
95%
96%
97%
98%
100%


In [39]:
# LOAD DATASET
feature_data =pd.read_csv("/content/drive/MyDrive/Dataset TA/PIR last 30 list dataset train 2 hari.csv")

# ADD LABEL ENCODER
label_encoder = LabelEncoder()
feature_data['Gerakan'] = label_encoder.fit_transform(feature_data['Gerakan'])
datas_labels = label_encoder.classes_

In [40]:
x = feature_data.drop('Gerakan',axis=1)
y = feature_data['Gerakan']

# RESHAPE DATASET
x = x.values.reshape(-1,30,5)   #INPUT 4
y = y.values.reshape((-1,1))
# x = x.values.reshape(-1,150,1)


# SPLIT DATASET
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

# CHECK THE SHAPE
print(x_train.shape , y_train.shape)
print(x_test.shape , y_test.shape)

# SET TOTAL FOLD
total_fold = 5

(39146, 30, 5) (39146, 1)
(16777, 30, 5) (16777, 1)


In [ ]:
# import tensorflow_decision_forests as tfdf
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
import statistics as st
# https://www.kaggle.com/code/pavansanagapati/ensemble-learning-techniques-tutorial
# https://www.analyticsvidhya.com/blog/2018/06/comprehensive-guide-for-ensemble-models/

In [ ]:
rf = RandomForestClassifier()
et = ExtraTreesClassifier()
knn = KNeighborsClassifier()
svc = SVC()
rg = RidgeClassifier()
clf_array = [rf, et, knn, svc, rg]

# **Data Train**

### Max Voting

In [ ]:
#MAX VOTING BEFORE TUNING

fold_number = 0

kf = KFold(n_splits = total_fold, shuffle=True)                                            
cvscores = []

for train_index, test_index in kf.split(x_train):
    x_train_fold, x_test_fold = x_train[train_index], x_train[test_index]                             
    y_train_fold, y_test_fold = y_train[train_index], y_train[test_index] 
    x_train_fold = x_train_fold.reshape(-1,150)
    x_test_fold = x_test_fold.reshape(-1,150)
    model1 = LogisticRegression(solver="lbfgs",random_state=1)
    model2 = DecisionTreeClassifier(criterion="gini",random_state=1)
    model = VotingClassifier(estimators=[('lr', model1), ('dt', model2)], voting='hard')
    model.fit(x_train_fold,y_train_fold)
    cvscores.append(model.score(x_test_fold, y_test_fold) * 100)
    fold_number+=1
    fold_name = "drive/MyDrive/Dataset TA/Max Voting Model/max_voting_before_tuning_fold" + str(fold_number) + ".sav"
    pickle.dump(model, open(fold_name, 'wb'))

print("Model Accuracy List: " , cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

filename = "drive/MyDrive/Dataset TA/Max Voting Model/max_voting_before_tuning.sav"
pickle.dump(model, open(filename, 'wb'))

/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regressio

Model Accuracy List:  [92.03065134099617, 92.18290969472474, 91.37820922212288, 91.97854132073063, 91.56980457274237]
91.83% (+/- 0.30%)


In [ ]:
#MAX VOTING AFTER TUNING

fold_number = 0

kf = KFold(n_splits = total_fold, shuffle=True)                                            
cvscores = []

for train_index, test_index in kf.split(x_train):
    x_train_fold, x_test_fold = x_train[train_index], x_train[test_index]                             
    y_train_fold, y_test_fold = y_train[train_index], y_train[test_index] 
    x_train_fold = x_train_fold.reshape(-1,150)
    x_test_fold = x_test_fold.reshape(-1,150)
    model1 = LogisticRegression(solver="liblinear", random_state=1)         #CHANGE SOLVER
    model2 = DecisionTreeClassifier(criterion = "entropy",random_state=1)   #CHANGE CRITERION
    model = VotingClassifier(estimators=[('lr', model1), ('dt', model2)], voting='hard')
    model.fit(x_train_fold,y_train_fold)
    cvscores.append(model.score(x_test_fold, y_test_fold) * 100)
    fold_number+=1
    fold_name = "drive/MyDrive/Dataset TA/Max Voting Model/max_voting_after_tuning_fold" + str(fold_number) + ".sav"
    pickle.dump(model, open(fold_name, 'wb'))

print("Model Accuracy List: " , cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

filename = "drive/MyDrive/Dataset TA/Max Voting Model/max_voting_after_tuning.sav"
pickle.dump(model, open(filename, 'wb'))

/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/

Model Accuracy List:  [92.18390804597702, 92.34895899859497, 91.97854132073063, 92.37450504534424, 92.71937667645932]
92.32% (+/- 0.24%)


### Bagging

In [ ]:
#BAGGING BEFORE TUNING

fold_number = 0

kf = KFold(n_splits = total_fold, shuffle=True)                                            
cvscores = []

for train_index, test_index in kf.split(x_train):
    x_train_fold, x_test_fold = x_train[train_index], x_train[test_index]                             
    y_train_fold, y_test_fold = y_train[train_index], y_train[test_index] 
    x_train_fold = x_train_fold.reshape(-1,150)
    x_test_fold = x_test_fold.reshape(-1,150)
    # Create classifiers
    rf = RandomForestClassifier()
    et = ExtraTreesClassifier()
    knn = KNeighborsClassifier()
    svc = SVC()
    rg = RidgeClassifier()
    clf_array = [rf, et, knn, svc, rg]
    for clf in clf_array:
        vanilla_scores = cross_val_score(clf, x_train_fold, y_train_fold, cv=10, n_jobs=-1)
        bagging_clf = BaggingClassifier(clf,max_samples=0.4, max_features=10, n_estimators = 10)    
        bagging_scores = cross_val_score(bagging_clf, x_train_fold, y_train_fold, cv=10,n_jobs=-1)
        
        print ("Mean of: {1:.3f}, std: (+/-) {2:.3f} [{0}]".format(clf.__class__.__name__,vanilla_scores.mean(), vanilla_scores.std()))
        print ("Mean of: {1:.3f}, std: (+/-) {2:.3f} [Bagging {0}]\n".format(clf.__class__.__name__,bagging_scores.mean(), bagging_scores.std()))
    clf = [rf, et, knn, svc, rg]
    eclf = VotingClassifier(estimators=[('Random Forests', rf), ('Extra Trees', et), ('KNeighbors', knn), ('SVC', svc), ('Ridge Classifier', rg)], voting='hard')
    eclf.fit(x_train_fold,y_train_fold)
    cvscores.append(eclf.score(x_test_fold, y_test_fold) * 100)
    fold_number+=1
    fold_name = "drive/MyDrive/Dataset TA/Bagging Model/bagging_before_tuning_fold" + str(fold_number) + ".sav"
    pickle.dump(eclf, open(fold_name, 'wb'))

print("Model Accuracy List: " , cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

filename = "drive/MyDrive/Dataset TA/Bagging Model/bagging_before_tuning.sav"
pickle.dump(eclf, open(filename, 'wb'))

Mean of: 0.957, std: (+/-) 0.004 [RandomForestClassifier]
Mean of: 0.694, std: (+/-) 0.036 [Bagging RandomForestClassifier]

Mean of: 0.955, std: (+/-) 0.003 [ExtraTreesClassifier]
Mean of: 0.667, std: (+/-) 0.020 [Bagging ExtraTreesClassifier]

Mean of: 0.708, std: (+/-) 0.006 [KNeighborsClassifier]
Mean of: 0.615, std: (+/-) 0.031 [Bagging KNeighborsClassifier]

Mean of: 0.709, std: (+/-) 0.006 [SVC]
Mean of: 0.579, std: (+/-) 0.035 [Bagging SVC]

Mean of: 0.949, std: (+/-) 0.003 [RidgeClassifier]
Mean of: 0.583, std: (+/-) 0.014 [Bagging RidgeClassifier]



/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean of: 0.957, std: (+/-) 0.003 [RandomForestClassifier]
Mean of: 0.687, std: (+/-) 0.037 [Bagging RandomForestClassifier]

Mean of: 0.955, std: (+/-) 0.003 [ExtraTreesClassifier]
Mean of: 0.690, std: (+/-) 0.022 [Bagging ExtraTreesClassifier]

Mean of: 0.713, std: (+/-) 0.006 [KNeighborsClassifier]
Mean of: 0.616, std: (+/-) 0.017 [Bagging KNeighborsClassifier]

Mean of: 0.711, std: (+/-) 0.006 [SVC]
Mean of: 0.543, std: (+/-) 0.028 [Bagging SVC]

Mean of: 0.948, std: (+/-) 0.002 [RidgeClassifier]
Mean of: 0.594, std: (+/-) 0.018 [Bagging RidgeClassifier]



/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean of: 0.957, std: (+/-) 0.003 [RandomForestClassifier]
Mean of: 0.693, std: (+/-) 0.035 [Bagging RandomForestClassifier]

Mean of: 0.953, std: (+/-) 0.002 [ExtraTreesClassifier]
Mean of: 0.677, std: (+/-) 0.020 [Bagging ExtraTreesClassifier]

Mean of: 0.711, std: (+/-) 0.005 [KNeighborsClassifier]
Mean of: 0.614, std: (+/-) 0.014 [Bagging KNeighborsClassifier]

Mean of: 0.708, std: (+/-) 0.006 [SVC]
Mean of: 0.574, std: (+/-) 0.016 [Bagging SVC]

Mean of: 0.948, std: (+/-) 0.002 [RidgeClassifier]
Mean of: 0.594, std: (+/-) 0.012 [Bagging RidgeClassifier]



...

Mean of: 0.712, std: (+/-) 0.008 [KNeighborsClassifier]
Mean of: 0.629, std: (+/-) 0.017 [Bagging KNeighborsClassifier]

Mean of: 0.708, std: (+/-) 0.007 [SVC]
Mean of: 0.556, std: (+/-) 0.017 [Bagging SVC]

Mean of: 0.949, std: (+/-) 0.003 [RidgeClassifier]
Mean of: 0.591, std: (+/-) 0.015 [Bagging RidgeClassifier]



/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean of: 0.958, std: (+/-) 0.004 [RandomForestClassifier]
Mean of: 0.685, std: (+/-) 0.029 [Bagging RandomForestClassifier]

Mean of: 0.955, std: (+/-) 0.003 [ExtraTreesClassifier]
Mean of: 0.684, std: (+/-) 0.034 [Bagging ExtraTreesClassifier]

Mean of: 0.711, std: (+/-) 0.007 [KNeighborsClassifier]
Mean of: 0.622, std: (+/-) 0.023 [Bagging KNeighborsClassifier]

Mean of: 0.710, std: (+/-) 0.007 [SVC]
Mean of: 0.538, std: (+/-) 0.017 [Bagging SVC]

Mean of: 0.948, std: (+/-) 0.003 [RidgeClassifier]
Mean of: 0.592, std: (+/-) 0.019 [Bagging RidgeClassifier]



/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Model Accuracy List:  [95.41507024265645, 95.45280367863073, 95.46557670200536, 95.23566228126198, 95.31230042150978]
95.38% (+/- 0.09%)


In [ ]:
#BAGGING AFTER TUNING

fold_number = 0

kf = KFold(n_splits = total_fold, shuffle=True)                                            
cvscores = []

for train_index, test_index in kf.split(x_train):
    x_train_fold, x_test_fold = x_train[train_index], x_train[test_index]                             
    y_train_fold, y_test_fold = y_train[train_index], y_train[test_index] 
    x_train_fold = x_train_fold.reshape(-1,150)
    x_test_fold = x_test_fold.reshape(-1,150)
    # Create classifiers
    rf = RandomForestClassifier()
    et = ExtraTreesClassifier()
    knn = KNeighborsClassifier()
    svc = SVC()
    rg = RidgeClassifier()
    clf_array = [rf, et, knn, svc, rg]
    for clf in clf_array:
        vanilla_scores = cross_val_score(clf, x_train_fold, y_train_fold, cv=10, n_jobs=-1)
        bagging_clf = BaggingClassifier(clf,max_samples=0.2, max_features=5, n_estimators=5)        #CHANGE MAX_SAMPLES, MAX FEATURES, N_ESTIMATORS
        bagging_scores = cross_val_score(bagging_clf, x_train_fold, y_train_fold, cv=10,n_jobs=-1)
        
        print ("Mean of: {1:.3f}, std: (+/-) {2:.3f} [{0}]".format(clf.__class__.__name__,vanilla_scores.mean(), vanilla_scores.std()))
        print ("Mean of: {1:.3f}, std: (+/-) {2:.3f} [Bagging {0}]\n".format(clf.__class__.__name__,bagging_scores.mean(), bagging_scores.std()))
    clf = [rf, et, knn, svc, rg]
    eclf = VotingClassifier(estimators=[('Random Forests', rf), ('Extra Trees', et), ('KNeighbors', knn), ('SVC', svc), ('Ridge Classifier', rg)], voting='hard')
    eclf.fit(x_train_fold,y_train_fold)
    cvscores.append(eclf.score(x_test_fold, y_test_fold) * 100)
    fold_number+=1
    fold_name = "drive/MyDrive/Dataset TA/Bagging Model/bagging_after_tuning_fold" + str(fold_number) + ".sav"
    pickle.dump(eclf, open(fold_name, 'wb'))

print("Model Accuracy List: " , cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

filename = "drive/MyDrive/Dataset TA/Bagging Model/bagging_after_tuning.sav"
pickle.dump(eclf, open(filename, 'wb'))

Mean of: 0.957, std: (+/-) 0.005 [RandomForestClassifier]
Mean of: 0.576, std: (+/-) 0.031 [Bagging RandomForestClassifier]

Mean of: 0.953, std: (+/-) 0.005 [ExtraTreesClassifier]
Mean of: 0.572, std: (+/-) 0.025 [Bagging ExtraTreesClassifier]

Mean of: 0.710, std: (+/-) 0.005 [KNeighborsClassifier]
Mean of: 0.540, std: (+/-) 0.014 [Bagging KNeighborsClassifier]

Mean of: 0.708, std: (+/-) 0.006 [SVC]
Mean of: 0.551, std: (+/-) 0.019 [Bagging SVC]

Mean of: 0.948, std: (+/-) 0.004 [RidgeClassifier]
Mean of: 0.550, std: (+/-) 0.017 [Bagging RidgeClassifier]



/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean of: 0.958, std: (+/-) 0.003 [RandomForestClassifier]
Mean of: 0.591, std: (+/-) 0.036 [Bagging RandomForestClassifier]

Mean of: 0.954, std: (+/-) 0.004 [ExtraTreesClassifier]
Mean of: 0.592, std: (+/-) 0.028 [Bagging ExtraTreesClassifier]

Mean of: 0.709, std: (+/-) 0.005 [KNeighborsClassifier]
Mean of: 0.552, std: (+/-) 0.029 [Bagging KNeighborsClassifier]

Mean of: 0.706, std: (+/-) 0.006 [SVC]
Mean of: 0.533, std: (+/-) 0.015 [Bagging SVC]

Mean of: 0.949, std: (+/-) 0.004 [RidgeClassifier]
Mean of: 0.551, std: (+/-) 0.018 [Bagging RidgeClassifier]



/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean of: 0.956, std: (+/-) 0.004 [RandomForestClassifier]
Mean of: 0.597, std: (+/-) 0.047 [Bagging RandomForestClassifier]

Mean of: 0.952, std: (+/-) 0.004 [ExtraTreesClassifier]
Mean of: 0.578, std: (+/-) 0.024 [Bagging ExtraTreesClassifier]

Mean of: 0.710, std: (+/-) 0.006 [KNeighborsClassifier]
Mean of: 0.541, std: (+/-) 0.026 [Bagging KNeighborsClassifier]

Mean of: 0.707, std: (+/-) 0.007 [SVC]
Mean of: 0.540, std: (+/-) 0.024 [Bagging SVC]

Mean of: 0.948, std: (+/-) 0.005 [RidgeClassifier]
Mean of: 0.553, std: (+/-) 0.018 [Bagging RidgeClassifier]



...

Mean of: 0.957, std: (+/-) 0.005 [RandomForestClassifier]
Mean of: 0.565, std: (+/-) 0.024 [Bagging RandomForestClassifier]

Mean of: 0.953, std: (+/-) 0.004 [ExtraTreesClassifier]
Mean of: 0.586, std: (+/-) 0.025 [Bagging ExtraTreesClassifier]

Mean of: 0.714, std: (+/-) 0.004 [KNeighborsClassifier]
Mean of: 0.541, std: (+/-) 0.013 [Bagging KNeighborsClassifier]

Mean of: 0.707, std: (+/-) 0.005 [SVC]
Mean of: 0.548, std: (+/-) 0.024 [Bagging SVC]

Mean of: 0.949, std: (+/-) 0.005 [RidgeClassifier]
Mean of: 0.542, std: (+/-) 0.011 [Bagging RidgeClassifier]



/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean of: 0.956, std: (+/-) 0.004 [RandomForestClassifier]
Mean of: 0.588, std: (+/-) 0.029 [Bagging RandomForestClassifier]

Mean of: 0.953, std: (+/-) 0.003 [ExtraTreesClassifier]
Mean of: 0.578, std: (+/-) 0.025 [Bagging ExtraTreesClassifier]

Mean of: 0.716, std: (+/-) 0.006 [KNeighborsClassifier]
Mean of: 0.533, std: (+/-) 0.019 [Bagging KNeighborsClassifier]

Mean of: 0.708, std: (+/-) 0.004 [SVC]
Mean of: 0.542, std: (+/-) 0.020 [Bagging SVC]

Mean of: 0.949, std: (+/-) 0.004 [RidgeClassifier]
Mean of: 0.546, std: (+/-) 0.018 [Bagging RidgeClassifier]



/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Model Accuracy List:  [95.50446998722862, 94.95465576702006, 95.45280367863073, 95.44003065525611, 95.3250734448844]
95.34% (+/- 0.20%)


###Boosting

In [ ]:
#BOOSTING BEFORE TUNING

fold_number = 0

kf = KFold(n_splits = total_fold, shuffle=True)                                            
cvscores = []

for train_index, test_index in kf.split(x_train):
    x_train_fold, x_test_fold = x_train[train_index], x_train[test_index]                             
    y_train_fold, y_test_fold = y_train[train_index], y_train[test_index] 
    x_train_fold = x_train_fold.reshape(-1,150)
    x_test_fold = x_test_fold.reshape(-1,150)
    #BOOSTING
    ada_boost = AdaBoostClassifier(n_estimators=100, learning_rate=0.01, algorithm= "SAMME.R",random_state=1)   
    ada_boost.fit(x_train_fold, y_train_fold)
    ada_boost.score(x_test_fold,y_test_fold)

    grad_boost= GradientBoostingClassifier(loss = "exponential", criterion="friedman_mse",learning_rate=0.01,random_state=1)
    grad_boost.fit(x_train_fold, y_train_fold)
    grad_boost.score(x_test_fold,y_test_fold)

    xgb_boost=xgb.XGBClassifier(random_state=1,learning_rate=0.01)
    xgb_boost.fit(x_train_fold, y_train_fold)
    xgb_boost.score(x_test_fold,y_test_fold)

    x_train = x_train.reshape(-1,150)

    eclf = VotingClassifier(estimators=[('Ada Boost', ada_boost), ('Grad Boost', grad_boost), ('XG Boost', xgb_boost)], voting='hard')
    clf = [rf, et, knn, svc, rg]
    for clf, label in zip([ada_boost, grad_boost, xgb_boost,eclf], ['Ada Boost','Grad Boost','XG Boost','Ensemble']):
        scores = cross_val_score(clf, x_train, y_train, cv=10, scoring='accuracy')
        print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))
    eclf.fit(x_train_fold,y_train_fold)
    cvscores.append(eclf.score(x_test_fold, y_test_fold) * 100)
    fold_number+=1
    fold_name = "drive/MyDrive/Dataset TA/Boosting Model/boosting_before_tuning_fold" + str(fold_number) + ".sav"
    pickle.dump(eclf, open(fold_name, 'wb'))

print("Model Accuracy List: " , cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
x_train = x_train.reshape(-1,30,5)
filename = "drive/MyDrive/Dataset TA/Boosting Model/boosting_before_tuning.sav"
pickle.dump(eclf, open(filename, 'wb'))

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example u

Accuracy: 0.68 (+/- 0.01) [Ada Boost]


/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y =

Accuracy: 0.71 (+/- 0.01) [Grad Boost]


/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), fo

...

Accuracy: 0.71 (+/- 0.01) [XG Boost]


/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), fo

Accuracy: 0.71 (+/- 0.01) [Ensemble]


/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Model Accuracy List:  [70.17879948914431, 70.60927321496997, 70.3410397241027, 70.35381274747733, 71.43951973432111]
70.58% (+/- 0.45%)


In [ ]:
#BOOSTING AFTER TUNING

fold_number = 0

kf = KFold(n_splits = total_fold, shuffle=True)                                            
cvscores = []

for train_index, test_index in kf.split(x_train):
    x_train_fold, x_test_fold = x_train[train_index], x_train[test_index]                             
    y_train_fold, y_test_fold = y_train[train_index], y_train[test_index] 
    x_train_fold = x_train_fold.reshape(-1,150)
    x_test_fold = x_test_fold.reshape(-1,150)
    #BOOSTING
    ada_boost = AdaBoostClassifier(n_estimators=50, learning_rate=0.1, algorithm= "SAMME",random_state=1)         #CHANGE N_ESTIMATORS, LEARNING_RATE, ALGORITHM
    ada_boost.fit(x_train_fold, y_train_fold)
    ada_boost.score(x_test_fold,y_test_fold)

    grad_boost= GradientBoostingClassifier(loss = "deviance", criterion="squared_error",learning_rate=0.1,random_state=1)     #CHANGE LOSS, CRITERION, LEARNING RATE
    grad_boost.fit(x_train_fold, y_train_fold)
    grad_boost.score(x_test_fold,y_test_fold)

    xgb_boost=xgb.XGBClassifier(random_state=1,learning_rate=0.1)    #CHANGE LEARNING RATE
    xgb_boost.fit(x_train_fold, y_train_fold)
    xgb_boost.score(x_test_fold,y_test_fold)

    x_train = x_train.reshape(-1,150)

    eclf = VotingClassifier(estimators=[('Ada Boost', ada_boost), ('Grad Boost', grad_boost), ('XG Boost', xgb_boost)], voting='hard')
    clf = [rf, et, knn, svc, rg]
    for clf, label in zip([ada_boost, grad_boost, xgb_boost,eclf], ['Ada Boost','Grad Boost','XG Boost','Ensemble']):
        scores = cross_val_score(clf, x_train, y_train, cv=10, scoring='accuracy')
        print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))
    eclf.fit(x_train_fold,y_train_fold)
    cvscores.append(eclf.score(x_test_fold, y_test_fold) * 100)
    fold_number+=1
    fold_name = "drive/MyDrive/Dataset TA/Boosting Model/boosting_after_tuning_fold" + str(fold_number) + ".sav"
    pickle.dump(eclf, open(fold_name, 'wb'))

print("Model Accuracy List: " , cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
x_train = x_train.reshape(-1,30,5)
filename = "drive/MyDrive/Dataset TA/Boosting Model/boosting_after_tuning.sav"
pickle.dump(eclf, open(filename, 'wb'))

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example u

Accuracy: 0.54 (+/- 0.01) [Ada Boost]


/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y =

Accuracy: 0.93 (+/- 0.00) [Grad Boost]


/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), fo

...

Accuracy: 0.91 (+/- 0.01) [XG Boost]


/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), fo

Accuracy: 0.91 (+/- 0.01) [Ensemble]


/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Model Accuracy List:  [89.8595146871009, 88.90024268744412, 90.04981479116107, 92.32341295184571, 91.41652829224678]
90.51% (+/- 1.21%)


In [ ]:
file_max_voting_before_tuning = [("drive/MyDrive/Dataset TA/Max Voting Model/max_voting_before_tuning_fold" + str(fn+1) + ".sav") for fn in range(total_fold)]
loaded_max_voting_before_tuning = [(pickle.load(open(file_max_voting_before_tuning[lm], "rb"))) for lm in range(total_fold)]
x_test = x_test.reshape(-1,150)
y_pred_max_voting_before_tuning = [(loaded_max_voting_before_tuning[yp].predict(x_test)) for yp in range(total_fold)] 
y_pred_values_max_voting_before_tuning = [y_pred_max_voting_before_tuning[ypv] for ypv in range(total_fold)]
for ypvc in range(total_fold):
  y_pred_values_max_voting_before_tuning[ypvc] = np.reshape(y_pred_values_max_voting_before_tuning[ypvc],(-1,1))
result_max_voting_before_tuning = [(classification_report(y_test, y_pred_values_max_voting_before_tuning[res], output_dict = True)) for res in range(total_fold)]

file_bagging_before_tuning = [("drive/MyDrive/Dataset TA/Bagging Model/bagging_before_tuning_fold" + str(fn+1) + ".sav") for fn in range(total_fold)]
loaded_bagging_before_tuning = [(pickle.load(open(file_bagging_before_tuning[lm], "rb"))) for lm in range(total_fold)]
x_test = x_test.reshape(-1,150)
y_pred_bagging_before_tuning = [(loaded_bagging_before_tuning[yp].predict(x_test)) for yp in range(total_fold)] 
y_pred_values_bagging_before_tuning = [y_pred_bagging_before_tuning[ypv] for ypv in range(total_fold)]
for ypvc in range(total_fold):
  y_pred_values_bagging_before_tuning[ypvc] = np.reshape(y_pred_values_bagging_before_tuning[ypvc],(-1,1))
result_bagging_before_tuning = [(classification_report(y_test, y_pred_values_bagging_before_tuning[res], output_dict = True)) for res in range(total_fold)]

file_boosting_before_tuning = [("drive/MyDrive/Dataset TA/Boosting Model/boosting_before_tuning_fold" + str(fn+1) + ".sav") for fn in range(total_fold)]
loaded_boosting_before_tuning = [(pickle.load(open(file_boosting_before_tuning[lm], "rb"))) for lm in range(total_fold)]
x_test = x_test.reshape(-1,150)
y_pred_boosting_before_tuning = [(loaded_boosting_before_tuning[yp].predict(x_test)) for yp in range(total_fold)] 
y_pred_values_boosting_before_tuning = [y_pred_boosting_before_tuning[ypv] for ypv in range(total_fold)]
for ypvc in range(total_fold):
  y_pred_values_boosting_before_tuning[ypvc] = np.reshape(y_pred_values_boosting_before_tuning[ypvc],(-1,1))
result_boosting_before_tuning = [(classification_report(y_test, y_pred_values_boosting_before_tuning[res], output_dict = True)) for res in range(total_fold)]

file_max_voting_after_tuning = [("drive/MyDrive/Dataset TA/Max Voting Model/max_voting_after_tuning_fold" + str(fn+1) + ".sav") for fn in range(total_fold)]
loaded_max_voting_after_tuning = [(pickle.load(open(file_max_voting_after_tuning[lm], "rb"))) for lm in range(total_fold)]
x_test = x_test.reshape(-1,150)
y_pred_max_voting_after_tuning = [(loaded_max_voting_after_tuning[yp].predict(x_test)) for yp in range(total_fold)] 
y_pred_values_max_voting_after_tuning = [y_pred_max_voting_after_tuning[ypv] for ypv in range(total_fold)]
for ypvc in range(total_fold):
  y_pred_values_max_voting_after_tuning[ypvc] = np.reshape(y_pred_values_max_voting_after_tuning[ypvc],(-1,1))
result_max_voting_after_tuning = [(classification_report(y_test, y_pred_values_max_voting_after_tuning[res], output_dict = True)) for res in range(total_fold)]

file_bagging_after_tuning = [("drive/MyDrive/Dataset TA/Bagging Model/bagging_after_tuning_fold" + str(fn+1) + ".sav") for fn in range(total_fold)]
loaded_bagging_after_tuning = [(pickle.load(open(file_bagging_after_tuning[lm], "rb"))) for lm in range(total_fold)]
x_test = x_test.reshape(-1,150)
y_pred_bagging_after_tuning = [(loaded_bagging_after_tuning[yp].predict(x_test)) for yp in range(total_fold)] 
y_pred_values_bagging_after_tuning = [y_pred_bagging_after_tuning[ypv] for ypv in range(total_fold)]
for ypvc in range(total_fold):
  y_pred_values_bagging_after_tuning[ypvc] = np.reshape(y_pred_values_bagging_after_tuning[ypvc],(-1,1))
result_bagging_after_tuning = [(classification_report(y_test, y_pred_values_bagging_after_tuning[res], output_dict = True)) for res in range(total_fold)]

file_boosting_after_tuning = [("drive/MyDrive/Dataset TA/Boosting Model/boosting_after_tuning_fold" + str(fn+1) + ".sav") for fn in range(total_fold)]
loaded_boosting_after_tuning = [(pickle.load(open(file_boosting_after_tuning[lm], "rb"))) for lm in range(total_fold)]
x_test = x_test.reshape(-1,150)
y_pred_boosting_after_tuning = [(loaded_boosting_after_tuning[yp].predict(x_test)) for yp in range(total_fold)] 
y_pred_values_boosting_after_tuning = [y_pred_boosting_after_tuning[ypv] for ypv in range(total_fold)]
for ypvc in range(total_fold):
  y_pred_values_boosting_after_tuning[ypvc] = np.reshape(y_pred_values_boosting_after_tuning[ypvc],(-1,1))
result_boosting_after_tuning = [(classification_report(y_test, y_pred_values_boosting_after_tuning[res], output_dict = True)) for res in range(total_fold)]

In [ ]:
precision_list_before_tuning = {}
recall_list_before_tuning = {}
accuracy_list_before_tuning = {}
f1_list_before_tuning = {}
precision_list_after_tuning = {}
recall_list_after_tuning = {}
accuracy_list_after_tuning = {}
f1_list_after_tuning = {}

In [ ]:
for sco in range(total_fold):
  precision_list_before_tuning[(sco+1)] = [result_max_voting_before_tuning[sco]['weighted avg']['precision'] , result_bagging_before_tuning[sco]['weighted avg']['precision'], result_boosting_before_tuning[sco]['weighted avg']['precision']]
  recall_list_before_tuning[(sco+1)] = [result_max_voting_before_tuning[sco]['weighted avg']['recall'] , result_bagging_before_tuning[sco]['weighted avg']['recall'], result_boosting_before_tuning[sco]['weighted avg']['recall']]
  f1_list_before_tuning[(sco+1)] = [result_max_voting_before_tuning[sco]['weighted avg']['f1-score'] , result_bagging_before_tuning[sco]['weighted avg']['f1-score'], result_boosting_before_tuning[sco]['weighted avg']['f1-score']]
  accuracy_list_before_tuning[(sco+1)] = [result_max_voting_before_tuning[sco]['accuracy'] , result_bagging_before_tuning[sco]['accuracy'], result_boosting_before_tuning[sco]['accuracy']]
  precision_list_after_tuning[(sco+1)] = [result_max_voting_after_tuning[sco]['weighted avg']['precision'] , result_bagging_after_tuning[sco]['weighted avg']['precision'], result_boosting_after_tuning[sco]['weighted avg']['precision']]
  recall_list_after_tuning[(sco+1)] = [result_max_voting_after_tuning[sco]['weighted avg']['recall'] , result_bagging_after_tuning[sco]['weighted avg']['recall'], result_boosting_after_tuning[sco]['weighted avg']['recall']]
  f1_list_after_tuning[(sco+1)] = [result_max_voting_after_tuning[sco]['weighted avg']['f1-score'] , result_bagging_after_tuning[sco]['weighted avg']['f1-score'], result_boosting_after_tuning[sco]['weighted avg']['f1-score']]
  accuracy_list_after_tuning[(sco+1)] = [result_max_voting_after_tuning[sco]['accuracy'] , result_bagging_after_tuning[sco]['accuracy'], result_boosting_after_tuning[sco]['accuracy']]

In [ ]:
print("")
print("")
print("{:<5} {:<0}".format("","Accuracy K-Fold Before Tuning"))
print ("{:<8} {:<10} {:<15} {:<10}".format('Fold','MAX VOTING','BAGGING','BOOSTING'))
for k, v in accuracy_list_before_tuning.items():
    the_max_voting, the_bagging, the_boosting  = v
    print ("{:<8} {:<15} {:<10} {:<10}".format(k, format(the_max_voting,".3f"), format(the_bagging,".3f"), format(the_boosting,".3f") ))
print("")
print("")

print("")
print("")
print("{:<5} {:<0}".format("","Precision K-Fold Before Tuning"))
print ("{:<8} {:<10} {:<15} {:<10}".format('Fold','MAX VOTING','BAGGING','BOOSTING'))
for k, v in precision_list_before_tuning.items():
    the_max_voting, the_bagging, the_boosting  = v
    print ("{:<8} {:<15} {:<10} {:<10}".format(k, format(the_max_voting,".3f"), format(the_bagging,".3f"), format(the_boosting,".3f") ))
print("")
print("")

print("")
print("")
print("{:<5} {:<0}".format("","Recall K-Fold Before Tuning"))
print ("{:<8} {:<10} {:<15} {:<10}".format('Fold','MAX VOTING','BAGGING','BOOSTING'))
for k, v in recall_list_before_tuning.items():
    the_max_voting, the_bagging, the_boosting  = v
    print ("{:<8} {:<15} {:<10} {:<10}".format(k, format(the_max_voting,".3f"), format(the_bagging,".3f"), format(the_boosting,".3f") ))
print("")
print("")

print("")
print("")
print("{:<5} {:<0}".format("","F1-Score K-Fold Before Tuning"))
print ("{:<8} {:<10} {:<15} {:<10}".format('Fold','MAX VOTING','BAGGING','BOOSTING'))
for k, v in f1_list_before_tuning.items():
    the_max_voting, the_bagging, the_boosting  = v
    print ("{:<8} {:<15} {:<10} {:<10}".format(k, format(the_max_voting,".3f"), format(the_bagging,".3f"), format(the_boosting,".3f") ))
print("")
print("")

print("")
print("")
print("{:<5} {:<0}".format("","Accuracy K-Fold After Tuning"))
print ("{:<8} {:<10} {:<15} {:<10}".format('Fold','MAX VOTING','BAGGING','BOOSTING'))
for k, v in accuracy_list_after_tuning.items():
    the_max_voting, the_bagging, the_boosting  = v
    print ("{:<8} {:<15} {:<10} {:<10}".format(k, format(the_max_voting,".3f"), format(the_bagging,".3f"), format(the_boosting,".3f") ))
print("")
print("")

print("")
print("")
print("{:<5} {:<0}".format("","Precision K-Fold After Tuning"))
print ("{:<8} {:<10} {:<15} {:<10}".format('Fold','MAX VOTING','BAGGING','BOOSTING'))
for k, v in precision_list_after_tuning.items():
    the_max_voting, the_bagging, the_boosting  = v
    print ("{:<8} {:<15} {:<10} {:<10}".format(k, format(the_max_voting,".3f"), format(the_bagging,".3f"), format(the_boosting,".3f") ))
print("")
print("")

print("")
print("")
print("{:<5} {:<0}".format("","Recall K-Fold After Tuning"))
print ("{:<8} {:<10} {:<15} {:<10}".format('Fold','MAX VOTING','BAGGING','BOOSTING'))
for k, v in recall_list_after_tuning.items():
    the_max_voting, the_bagging, the_boosting  = v
    print ("{:<8} {:<15} {:<10} {:<10}".format(k, format(the_max_voting,".3f"), format(the_bagging,".3f"), format(the_boosting,".3f") ))
print("")
print("")

print("")
print("")
print("{:<5} {:<0}".format("","F1-Score K-Fold After Tuning"))
print ("{:<8} {:<10} {:<15} {:<10}".format('Fold','MAX VOTING','BAGGING','BOOSTING'))
for k, v in f1_list_after_tuning.items():
    the_max_voting, the_bagging, the_boosting  = v
    print ("{:<8} {:<15} {:<10} {:<10}".format(k, format(the_max_voting,".3f"), format(the_bagging,".3f"), format(the_boosting,".3f") ))
print("")
print("")



      Accuracy K-Fold Before Tuning
Fold     MAX VOTING BAGGING         BOOSTING  
1        0.929           0.969      0.706     
2        0.928           0.970      0.706     
3        0.928           0.969      0.706     
4        0.930           0.969      0.706     
5        0.929           0.967      0.706     




      Precision K-Fold Before Tuning
Fold     MAX VOTING BAGGING         BOOSTING  
1        0.936           0.969      0.811     
2        0.935           0.970      0.811     
3        0.935           0.969      0.810     
4        0.937           0.969      0.811     
5        0.935           0.968      0.810     




      Recall K-Fold Before Tuning
Fold     MAX VOTING BAGGING         BOOSTING  
1        0.929           0.969      0.706     
2        0.928           0.970      0.706     
3        0.928           0.969      0.706     
4        0.930           0.969      0.706     
5        0.929           0.967      0.706     




      F1-Score K-Fold Before Tuni

# **Data Train**

### Max Voting

In [ ]:
#MAX VOTING 30% DATA BEFORE TUNING

fold_number = 0

kf = KFold(n_splits = total_fold, shuffle=True)                                            
cvscores = []

for train_index, test_index in kf.split(x_test):
    x_train_fold, x_test_fold = x_test[train_index], x_test[test_index]                             
    y_train_fold, y_test_fold = y_test[train_index], y_test[test_index] 
    x_train_fold = x_train_fold.reshape(-1,150)
    x_test_fold = x_test_fold.reshape(-1,150)
    model1 = LogisticRegression(solver="liblinear", random_state=1)         
    model2 = DecisionTreeClassifier(criterion = "entropy",random_state=1)  
    model = VotingClassifier(estimators=[('lr', model1), ('dt', model2)], voting='hard')
    model.fit(x_train_fold,y_train_fold)
    cvscores.append(model.score(x_test_fold, y_test_fold) * 100)
    fold_number+=1
    fold_name = "drive/MyDrive/Dataset TA/Max Voting Model/max_voting_30_before_data_fold" + str(fold_number) + ".sav"
    pickle.dump(model, open(fold_name, 'wb'))

print("Model Accuracy List: " , cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

filename = "drive/MyDrive/Dataset TA/Max Voting Model/max_voting_30_before_data.sav"
pickle.dump(model, open(filename, 'wb'))

/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/

Model Accuracy List:  [92.93802145411205, 92.69964243146603, 93.56184798807749, 93.71087928464978, 92.66766020864382]
93.12% (+/- 0.44%)


In [ ]:
#MAX VOTING 30% DATA AFTER TUNING

fold_number = 0

kf = KFold(n_splits = total_fold, shuffle=True)                                            
cvscores = []

for train_index, test_index in kf.split(x_train):
    x_train_fold, x_test_fold = x_train[train_index], x_train[test_index]                             
    y_train_fold, y_test_fold = y_train[train_index], y_train[test_index] 
    x_train_fold = x_train_fold.reshape(-1,150)
    x_test_fold = x_test_fold.reshape(-1,150)
    model1 = LogisticRegression(solver="saga", random_state=1)         #CHANGE SOLVER
    model2 = DecisionTreeClassifier(criterion ="gini",random_state=1)   #CHANGE CRITERION
    model = VotingClassifier(estimators=[('lr', model1), ('dt', model2)], voting='hard')
    model.fit(x_train_fold,y_train_fold)
    cvscores.append(model.score(x_test_fold, y_test_fold) * 100)
    fold_number+=1
    fold_name = "drive/MyDrive/Dataset TA/Max Voting Model/max_voting_30_after_tuning_fold" + str(fold_number) + ".sav"
    pickle.dump(model, open(fold_name, 'wb'))

print("Model Accuracy List: " , cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

filename = "drive/MyDrive/Dataset TA/Max Voting Model/max_voting_30_after_tuning.sav"
pickle.dump(model, open(filename, 'wb'))

/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-pac

Model Accuracy List:  [74.21455938697318, 74.82437092859881, 74.51781836760762, 74.58168348448078, 74.77327883510027]
74.58% (+/- 0.22%)


### Bagging

In [ ]:
#BAGGING 30% DATA BEFORE TUNING

fold_number = 0

kf = KFold(n_splits = total_fold, shuffle=True)                                            
cvscores = []

for train_index, test_index in kf.split(x_test):
    x_train_fold, x_test_fold = x_test[train_index], x_test[test_index]                             
    y_train_fold, y_test_fold = y_test[train_index], y_test[test_index] 
    x_train_fold = x_train_fold.reshape(-1,150)
    x_test_fold = x_test_fold.reshape(-1,150)
    # Create classifiers
    rf = RandomForestClassifier()
    et = ExtraTreesClassifier()
    knn = KNeighborsClassifier()
    svc = SVC()
    rg = RidgeClassifier()
    clf_array = [rf, et, knn, svc, rg]
    for clf in clf_array:
        vanilla_scores = cross_val_score(clf, x_train_fold, y_train_fold, cv=10, n_jobs=-1)
        bagging_clf = BaggingClassifier(clf,max_samples=0.2, max_features=5, n_estimators=5)
        bagging_scores = cross_val_score(bagging_clf, x_train_fold, y_train_fold, cv=10,n_jobs=-1)
        
        print ("Mean of: {1:.3f}, std: (+/-) {2:.3f} [{0}]".format(clf.__class__.__name__,vanilla_scores.mean(), vanilla_scores.std()))
        print ("Mean of: {1:.3f}, std: (+/-) {2:.3f} [Bagging {0}]\n".format(clf.__class__.__name__,bagging_scores.mean(), bagging_scores.std()))
    clf = [rf, et, knn, svc, rg]
    eclf = VotingClassifier(estimators=[('Random Forests', rf), ('Extra Trees', et), ('KNeighbors', knn), ('SVC', svc), ('Ridge Classifier', rg)], voting='hard')
    eclf.fit(x_train_fold,y_train_fold)
    cvscores.append(eclf.score(x_test_fold, y_test_fold) * 100)
    fold_number+=1
    fold_name = "drive/MyDrive/Dataset TA/Bagging Model/bagging_30_before_data_fold" + str(fold_number) + ".sav"
    pickle.dump(eclf, open(fold_name, 'wb'))

print("Model Accuracy List: " , cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

filename = "drive/MyDrive/Dataset TA/Bagging Model/bagging_30_before_data.sav"
pickle.dump(eclf, open(filename, 'wb'))

Mean of: 0.956, std: (+/-) 0.005 [RandomForestClassifier]
Mean of: 0.576, std: (+/-) 0.022 [Bagging RandomForestClassifier]

Mean of: 0.954, std: (+/-) 0.006 [ExtraTreesClassifier]
Mean of: 0.568, std: (+/-) 0.033 [Bagging ExtraTreesClassifier]

Mean of: 0.700, std: (+/-) 0.014 [KNeighborsClassifier]
Mean of: 0.539, std: (+/-) 0.021 [Bagging KNeighborsClassifier]

Mean of: 0.705, std: (+/-) 0.012 [SVC]
Mean of: 0.524, std: (+/-) 0.022 [Bagging SVC]

Mean of: 0.950, std: (+/-) 0.004 [RidgeClassifier]
Mean of: 0.554, std: (+/-) 0.020 [Bagging RidgeClassifier]



/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean of: 0.954, std: (+/-) 0.004 [RandomForestClassifier]
Mean of: 0.582, std: (+/-) 0.025 [Bagging RandomForestClassifier]

Mean of: 0.950, std: (+/-) 0.004 [ExtraTreesClassifier]
Mean of: 0.597, std: (+/-) 0.046 [Bagging ExtraTreesClassifier]

Mean of: 0.700, std: (+/-) 0.014 [KNeighborsClassifier]
Mean of: 0.532, std: (+/-) 0.028 [Bagging KNeighborsClassifier]

Mean of: 0.706, std: (+/-) 0.013 [SVC]
Mean of: 0.532, std: (+/-) 0.023 [Bagging SVC]

Mean of: 0.949, std: (+/-) 0.004 [RidgeClassifier]
Mean of: 0.559, std: (+/-) 0.023 [Bagging RidgeClassifier]



/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean of: 0.954, std: (+/-) 0.004 [RandomForestClassifier]
Mean of: 0.563, std: (+/-) 0.022 [Bagging RandomForestClassifier]

Mean of: 0.953, std: (+/-) 0.004 [ExtraTreesClassifier]
Mean of: 0.581, std: (+/-) 0.040 [Bagging ExtraTreesClassifier]

Mean of: 0.703, std: (+/-) 0.011 [KNeighborsClassifier]
Mean of: 0.526, std: (+/-) 0.024 [Bagging KNeighborsClassifier]

Mean of: 0.708, std: (+/-) 0.011 [SVC]
Mean of: 0.536, std: (+/-) 0.017 [Bagging SVC]

Mean of: 0.949, std: (+/-) 0.005 [RidgeClassifier]
Mean of: 0.552, std: (+/-) 0.015 [Bagging RidgeClassifier]



...

Mean of: 0.954, std: (+/-) 0.004 [RandomForestClassifier]
Mean of: 0.581, std: (+/-) 0.027 [Bagging RandomForestClassifier]

Mean of: 0.953, std: (+/-) 0.004 [ExtraTreesClassifier]
Mean of: 0.581, std: (+/-) 0.028 [Bagging ExtraTreesClassifier]

Mean of: 0.701, std: (+/-) 0.010 [KNeighborsClassifier]
Mean of: 0.544, std: (+/-) 0.017 [Bagging KNeighborsClassifier]

Mean of: 0.707, std: (+/-) 0.013 [SVC]
Mean of: 0.543, std: (+/-) 0.037 [Bagging SVC]

Mean of: 0.947, std: (+/-) 0.006 [RidgeClassifier]
Mean of: 0.542, std: (+/-) 0.023 [Bagging RidgeClassifier]



/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean of: 0.953, std: (+/-) 0.006 [RandomForestClassifier]
Mean of: 0.563, std: (+/-) 0.015 [Bagging RandomForestClassifier]

Mean of: 0.951, std: (+/-) 0.005 [ExtraTreesClassifier]
Mean of: 0.563, std: (+/-) 0.023 [Bagging ExtraTreesClassifier]

Mean of: 0.701, std: (+/-) 0.013 [KNeighborsClassifier]
Mean of: 0.544, std: (+/-) 0.018 [Bagging KNeighborsClassifier]

Mean of: 0.705, std: (+/-) 0.009 [SVC]
Mean of: 0.532, std: (+/-) 0.029 [Bagging SVC]

Mean of: 0.948, std: (+/-) 0.005 [RidgeClassifier]
Mean of: 0.546, std: (+/-) 0.012 [Bagging RidgeClassifier]



/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Model Accuracy List:  [95.1430274135876, 94.87485101311086, 95.1117734724292, 95.29061102831594, 95.0521609538003]
95.09% (+/- 0.14%)


In [ ]:
#BAGGING 30% DATA AFTER TUNING

fold_number = 0

kf = KFold(n_splits = total_fold, shuffle=True)                                            
cvscores = []

for train_index, test_index in kf.split(x_test):
    x_train_fold, x_test_fold = x_test[train_index], x_test[test_index]                             
    y_train_fold, y_test_fold = y_test[train_index], y_test[test_index] 
    x_train_fold = x_train_fold.reshape(-1,150)
    x_test_fold = x_test_fold.reshape(-1,150)
    # Create classifiers
    rf = RandomForestClassifier()
    et = ExtraTreesClassifier()
    knn = KNeighborsClassifier()
    svc = SVC()
    rg = RidgeClassifier()
    clf_array = [rf, et, knn, svc, rg]
    for clf in clf_array:
        vanilla_scores = cross_val_score(clf, x_train_fold, y_train_fold, cv=10, n_jobs=-1)
        bagging_clf = BaggingClassifier(clf,max_samples=0.6, max_features=10, n_estimators=10)
        bagging_scores = cross_val_score(bagging_clf, x_train_fold, y_train_fold, cv=10,n_jobs=-1)
        
        print ("Mean of: {1:.3f}, std: (+/-) {2:.3f} [{0}]".format(clf.__class__.__name__,vanilla_scores.mean(), vanilla_scores.std()))
        print ("Mean of: {1:.3f}, std: (+/-) {2:.3f} [Bagging {0}]\n".format(clf.__class__.__name__,bagging_scores.mean(), bagging_scores.std()))
    clf = [rf, et, knn, svc, rg]
    eclf = VotingClassifier(estimators=[('Random Forests', rf), ('Extra Trees', et), ('KNeighbors', knn), ('SVC', svc), ('Ridge Classifier', rg)], voting='hard')
    eclf.fit(x_train_fold,y_train_fold)
    cvscores.append(eclf.score(x_test_fold, y_test_fold) * 100)
    fold_number+=1
    fold_name = "drive/MyDrive/Dataset TA/Bagging Model/bagging_30_after_data_fold" + str(fold_number) + ".sav"
    pickle.dump(eclf, open(fold_name, 'wb'))

print("Model Accuracy List: " , cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

filename = "drive/MyDrive/Dataset TA/Bagging Model/bagging_30_after_data.sav"
pickle.dump(eclf, open(filename, 'wb'))

Mean of: 0.956, std: (+/-) 0.004 [RandomForestClassifier]
Mean of: 0.707, std: (+/-) 0.037 [Bagging RandomForestClassifier]

Mean of: 0.953, std: (+/-) 0.006 [ExtraTreesClassifier]
Mean of: 0.669, std: (+/-) 0.019 [Bagging ExtraTreesClassifier]

Mean of: 0.702, std: (+/-) 0.014 [KNeighborsClassifier]
Mean of: 0.617, std: (+/-) 0.031 [Bagging KNeighborsClassifier]

Mean of: 0.706, std: (+/-) 0.011 [SVC]
Mean of: 0.533, std: (+/-) 0.023 [Bagging SVC]

Mean of: 0.950, std: (+/-) 0.006 [RidgeClassifier]
Mean of: 0.583, std: (+/-) 0.008 [Bagging RidgeClassifier]



/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean of: 0.954, std: (+/-) 0.005 [RandomForestClassifier]
Mean of: 0.684, std: (+/-) 0.026 [Bagging RandomForestClassifier]

Mean of: 0.951, std: (+/-) 0.004 [ExtraTreesClassifier]
Mean of: 0.669, std: (+/-) 0.033 [Bagging ExtraTreesClassifier]

Mean of: 0.702, std: (+/-) 0.011 [KNeighborsClassifier]
Mean of: 0.612, std: (+/-) 0.024 [Bagging KNeighborsClassifier]

Mean of: 0.707, std: (+/-) 0.011 [SVC]
Mean of: 0.543, std: (+/-) 0.029 [Bagging SVC]

Mean of: 0.947, std: (+/-) 0.005 [RidgeClassifier]
Mean of: 0.594, std: (+/-) 0.021 [Bagging RidgeClassifier]



/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean of: 0.954, std: (+/-) 0.003 [RandomForestClassifier]
Mean of: 0.691, std: (+/-) 0.028 [Bagging RandomForestClassifier]

Mean of: 0.953, std: (+/-) 0.003 [ExtraTreesClassifier]
Mean of: 0.673, std: (+/-) 0.022 [Bagging ExtraTreesClassifier]

Mean of: 0.701, std: (+/-) 0.012 [KNeighborsClassifier]
Mean of: 0.616, std: (+/-) 0.022 [Bagging KNeighborsClassifier]

Mean of: 0.704, std: (+/-) 0.013 [SVC]
Mean of: 0.555, std: (+/-) 0.031 [Bagging SVC]

Mean of: 0.949, std: (+/-) 0.003 [RidgeClassifier]
Mean of: 0.581, std: (+/-) 0.017 [Bagging RidgeClassifier]



...

Mean of: 0.953, std: (+/-) 0.005 [RandomForestClassifier]
Mean of: 0.677, std: (+/-) 0.028 [Bagging RandomForestClassifier]

Mean of: 0.950, std: (+/-) 0.006 [ExtraTreesClassifier]
Mean of: 0.680, std: (+/-) 0.032 [Bagging ExtraTreesClassifier]

Mean of: 0.706, std: (+/-) 0.010 [KNeighborsClassifier]
Mean of: 0.614, std: (+/-) 0.026 [Bagging KNeighborsClassifier]

Mean of: 0.707, std: (+/-) 0.011 [SVC]
Mean of: 0.561, std: (+/-) 0.044 [Bagging SVC]

Mean of: 0.947, std: (+/-) 0.005 [RidgeClassifier]
Mean of: 0.593, std: (+/-) 0.022 [Bagging RidgeClassifier]



/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean of: 0.954, std: (+/-) 0.004 [RandomForestClassifier]
Mean of: 0.683, std: (+/-) 0.023 [Bagging RandomForestClassifier]

Mean of: 0.952, std: (+/-) 0.005 [ExtraTreesClassifier]
Mean of: 0.669, std: (+/-) 0.026 [Bagging ExtraTreesClassifier]

Mean of: 0.705, std: (+/-) 0.013 [KNeighborsClassifier]
Mean of: 0.628, std: (+/-) 0.024 [Bagging KNeighborsClassifier]

Mean of: 0.709, std: (+/-) 0.011 [SVC]
Mean of: 0.535, std: (+/-) 0.026 [Bagging SVC]

Mean of: 0.949, std: (+/-) 0.005 [RidgeClassifier]
Mean of: 0.583, std: (+/-) 0.025 [Bagging RidgeClassifier]



/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Model Accuracy List:  [94.90464839094159, 95.44100119189511, 95.08196721311475, 95.7973174366617, 94.96274217585693]
95.24% (+/- 0.34%)


### Boosing

In [ ]:
#BOOSTING 30% DATA BEFORE TUNING

fold_number = 0

kf = KFold(n_splits = total_fold, shuffle=True)                                            
cvscores = []

for train_index, test_index in kf.split(x_test):
    x_train_fold, x_test_fold = x_test[train_index], x_test[test_index]                             
    y_train_fold, y_test_fold = y_test[train_index], y_test[test_index] 
    x_train_fold = x_train_fold.reshape(-1,150)
    x_test_fold = x_test_fold.reshape(-1,150)
    #BOOSTING
    ada_boost = AdaBoostClassifier(n_estimators=100, learning_rate=0.01, algorithm= "SAMME.R",random_state=1)         
    ada_boost.fit(x_train_fold, y_train_fold)
    ada_boost.score(x_test_fold,y_test_fold)

    grad_boost= GradientBoostingClassifier(loss = "exponential", criterion="friedman_mse",learning_rate=0.01,random_state=1)    
    grad_boost.fit(x_train_fold, y_train_fold)
    grad_boost.score(x_test_fold,y_test_fold)

    xgb_boost=xgb.XGBClassifier(random_state=1,learning_rate=0.01)    
    xgb_boost.fit(x_train_fold, y_train_fold)
    xgb_boost.score(x_test_fold,y_test_fold)

    x_test = x_test.reshape(-1,150)

    eclf = VotingClassifier(estimators=[('Ada Boost', ada_boost), ('Grad Boost', grad_boost), ('XG Boost', xgb_boost)], voting='hard')
    clf = [rf, et, knn, svc, rg]
    for clf, label in zip([ada_boost, grad_boost, xgb_boost,eclf], ['Ada Boost','Grad Boost','XG Boost','Ensemble']):
        scores = cross_val_score(clf, x_test, y_test, cv=10, scoring='accuracy')
        print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))
    eclf.fit(x_train_fold,y_train_fold)
    cvscores.append(eclf.score(x_test_fold, y_test_fold) * 100)
    fold_number+=1
    fold_name = "drive/MyDrive/Dataset TA/Boosting Model/boosting_30_before_data_fold" + str(fold_number) + ".sav"
    pickle.dump(eclf, open(fold_name, 'wb'))

print("Model Accuracy List: " , cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
x_test = x_test.reshape(-1,30,5)
filename = "drive/MyDrive/Dataset TA/Boosting Model/boosting_30_before_data.sav"
pickle.dump(eclf, open(filename, 'wb'))

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example u

Accuracy: 0.71 (+/- 0.01) [Ada Boost]


/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y =

Accuracy: 0.71 (+/- 0.01) [Grad Boost]


/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), fo

...

Accuracy: 0.71 (+/- 0.01) [XG Boost]


/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), fo

Accuracy: 0.71 (+/- 0.01) [Ensemble]


/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Model Accuracy List:  [70.70917759237187, 70.41120381406436, 70.34277198211625, 70.3725782414307, 70.3725782414307]
70.44% (+/- 0.14%)


In [41]:
#BOOSTING 30% DATA AFTER TUNING

fold_number = 0

kf = KFold(n_splits = total_fold, shuffle=True)                                            
cvscores = []

for train_index, test_index in kf.split(x_test):
    x_train_fold, x_test_fold = x_test[train_index], x_test[test_index]                             
    y_train_fold, y_test_fold = y_test[train_index], y_test[test_index] 
    x_train_fold = x_train_fold.reshape(-1,150)
    x_test_fold = x_test_fold.reshape(-1,150)
    #BOOSTING
    ada_boost = AdaBoostClassifier(n_estimators=50, learning_rate=0.1, algorithm= "SAMME",random_state=1)         
    ada_boost.fit(x_train_fold, y_train_fold)
    ada_boost.score(x_test_fold,y_test_fold)

    grad_boost= GradientBoostingClassifier(loss = "deviance", criterion="squared_error",learning_rate=0.1,random_state=1)    
    grad_boost.fit(x_train_fold, y_train_fold)
    grad_boost.score(x_test_fold,y_test_fold)

    xgb_boost=xgb.XGBClassifier(random_state=1,learning_rate=0.01)    
    xgb_boost.fit(x_train_fold, y_train_fold)
    xgb_boost.score(x_test_fold,y_test_fold)

    x_test = x_test.reshape(-1,150)

    eclf = VotingClassifier(estimators=[('Ada Boost', ada_boost), ('Grad Boost', grad_boost), ('XG Boost', xgb_boost)], voting='hard')
    clf = [rf, et, knn, svc, rg]
    for clf, label in zip([ada_boost, grad_boost, xgb_boost,eclf], ['Ada Boost','Grad Boost','XG Boost','Ensemble']):
        scores = cross_val_score(clf, x_test, y_test, cv=10, scoring='accuracy')
        print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))
    eclf.fit(x_train_fold,y_train_fold)
    cvscores.append(eclf.score(x_test_fold, y_test_fold) * 100)
    fold_number+=1
    fold_name = "drive/MyDrive/Dataset TA/Boosting Model/boosting_30_after_data_fold" + str(fold_number) + ".sav"
    pickle.dump(eclf, open(fold_name, 'wb'))

print("Model Accuracy List: " , cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
x_test = x_test.reshape(-1,30,5)
filename = "drive/MyDrive/Dataset TA/Boosting Model/boosting_30_after_data.sav"
pickle.dump(eclf, open(filename, 'wb'))

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example u

Accuracy: 0.51 (+/- 0.01) [Ada Boost]


/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y =

Accuracy: 0.93 (+/- 0.01) [Grad Boost]


/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), fo

...

Accuracy: 0.70 (+/- 0.01) [XG Boost]


/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), fo

Accuracy: 0.81 (+/- 0.12) [Ensemble]


/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Model Accuracy List:  [93.56376638855781, 70.58998808104887, 69.83606557377048, 92.84649776453055, 93.62146050670641]
84.09% (+/- 11.34%)


In [45]:
file_max_voting_30_before_data = [("drive/MyDrive/Dataset TA/Max Voting Model/max_voting_30_before_data_fold" + str(fn+1) + ".sav") for fn in range(total_fold)]
loaded_max_voting_30_before_data = [(pickle.load(open(file_max_voting_30_before_data[lm], "rb"))) for lm in range(total_fold)]
x_test = x_test.reshape(-1,150)
y_pred_max_voting_30_before_data = [(loaded_max_voting_30_before_data[yp].predict(x_test)) for yp in range(total_fold)] 
y_pred_values_max_voting_30_before_data = [y_pred_max_voting_30_before_data[ypv] for ypv in range(total_fold)]
for ypvc in range(total_fold):
  y_pred_values_max_voting_30_before_data[ypvc] = np.reshape(y_pred_values_max_voting_30_before_data[ypvc],(-1,1))
result_max_voting_30_before_data = [(classification_report(y_test, y_pred_values_max_voting_30_before_data[res], output_dict = True)) for res in range(total_fold)]

file_bagging_30_before_data = [("drive/MyDrive/Dataset TA/Bagging Model/bagging_30_before_data_fold" + str(fn+1) + ".sav") for fn in range(total_fold)]
loaded_bagging_30_before_data = [(pickle.load(open(file_bagging_30_before_data[lm], "rb"))) for lm in range(total_fold)]
x_test = x_test.reshape(-1,150)
y_pred_bagging_30_before_data = [(loaded_bagging_30_before_data[yp].predict(x_test)) for yp in range(total_fold)] 
y_pred_values_bagging_30_before_data = [y_pred_bagging_30_before_data[ypv] for ypv in range(total_fold)]
for ypvc in range(total_fold):
  y_pred_values_bagging_30_before_data[ypvc] = np.reshape(y_pred_values_bagging_30_before_data[ypvc],(-1,1))
result_bagging_30_before_data = [(classification_report(y_test, y_pred_values_bagging_30_before_data[res], output_dict = True)) for res in range(total_fold)]

file_boosting_30_before_data = [("drive/MyDrive/Dataset TA/Boosting Model/boosting_30_before_data_fold" + str(fn+1) + ".sav") for fn in range(total_fold)]
loaded_boosting_30_before_data = [(pickle.load(open(file_boosting_30_before_data[lm], "rb"))) for lm in range(total_fold)]
x_test = x_test.reshape(-1,150)
y_pred_boosting_30_before_data = [(loaded_boosting_30_before_data[yp].predict(x_test)) for yp in range(total_fold)] 
y_pred_values_boosting_30_before_data = [y_pred_boosting_30_before_data[ypv] for ypv in range(total_fold)]
for ypvc in range(total_fold):
  y_pred_values_boosting_30_before_data[ypvc] = np.reshape(y_pred_values_boosting_30_before_data[ypvc],(-1,1))
result_boosting_30_before_data = [(classification_report(y_test, y_pred_values_boosting_30_before_data[res], output_dict = True)) for res in range(total_fold)]

file_max_voting_30_after_tuning = [("drive/MyDrive/Dataset TA/Max Voting Model/max_voting_30_after_tuning_fold" + str(fn+1) + ".sav") for fn in range(total_fold)]
loaded_max_voting_30_after_tuning = [(pickle.load(open(file_max_voting_30_after_tuning[lm], "rb"))) for lm in range(total_fold)]
x_test = x_test.reshape(-1,150)
y_pred_max_voting_30_after_tuning = [(loaded_max_voting_30_after_tuning[yp].predict(x_test)) for yp in range(total_fold)] 
y_pred_values_max_voting_30_after_tuning = [y_pred_max_voting_30_after_tuning[ypv] for ypv in range(total_fold)]
for ypvc in range(total_fold):
  y_pred_values_max_voting_30_after_tuning[ypvc] = np.reshape(y_pred_values_max_voting_30_after_tuning[ypvc],(-1,1))
result_max_voting_30_after_tuning = [(classification_report(y_test, y_pred_values_max_voting_30_after_tuning[res], output_dict = True)) for res in range(total_fold)]

file_bagging_30_after_data = [("drive/MyDrive/Dataset TA/Bagging Model/bagging_30_after_data_fold" + str(fn+1) + ".sav") for fn in range(total_fold)]
loaded_bagging_30_after_data = [(pickle.load(open(file_bagging_30_after_data[lm], "rb"))) for lm in range(total_fold)]
x_test = x_test.reshape(-1,150)
y_pred_bagging_30_after_data = [(loaded_bagging_30_after_data[yp].predict(x_test)) for yp in range(total_fold)] 
y_pred_values_bagging_30_after_data = [y_pred_bagging_30_after_data[ypv] for ypv in range(total_fold)]
for ypvc in range(total_fold):
  y_pred_values_bagging_30_after_data[ypvc] = np.reshape(y_pred_values_bagging_30_after_data[ypvc],(-1,1))
result_bagging_30_after_data = [(classification_report(y_test, y_pred_values_bagging_30_after_data[res], output_dict = True)) for res in range(total_fold)]

file_boosting_30_after_data = [("drive/MyDrive/Dataset TA/Boosting Model/boosting_30_after_data_fold" + str(fn+1) + ".sav") for fn in range(total_fold)]
loaded_boosting_30_after_data = [(pickle.load(open(file_boosting_30_after_data[lm], "rb"))) for lm in range(total_fold)]
x_test = x_test.reshape(-1,150)
y_pred_boosting_30_after_data = [(loaded_boosting_30_after_data[yp].predict(x_test)) for yp in range(total_fold)] 
y_pred_values_boosting_30_after_data = [y_pred_boosting_30_after_data[ypv] for ypv in range(total_fold)]
for ypvc in range(total_fold):
  y_pred_values_boosting_30_after_data[ypvc] = np.reshape(y_pred_values_boosting_30_after_data[ypvc],(-1,1))
result_boosting_30_after_data = [(classification_report(y_test, y_pred_values_boosting_30_after_data[res], output_dict = True)) for res in range(total_fold)]

x_train = x_train.reshape(-1,30,5)
x_test = x_test.reshape(-1,30,5)

In [46]:
precision_list_30_before_data = {}
recall_list_30_before_data = {}
accuracy_list_30_before_data = {}
f1_list_30_before_data = {}
precision_list_30_after_data = {}
recall_list_30_after_data = {}
accuracy_list_30_after_data = {}
f1_list_30_after_data = {}

In [52]:
for sco in range(total_fold):
  precision_list_30_before_data[(sco+1)] = [result_max_voting_30_before_data[sco]['weighted avg']['precision'] , result_bagging_30_before_data[sco]['weighted avg']['precision'], result_boosting_30_before_data[sco]['weighted avg']['precision']]
  recall_list_30_before_data[(sco+1)] = [result_max_voting_30_before_data[sco]['weighted avg']['recall'] , result_bagging_30_before_data[sco]['weighted avg']['recall'], result_boosting_30_before_data[sco]['weighted avg']['recall']]
  f1_list_30_before_data[(sco+1)] = [result_max_voting_30_before_data[sco]['weighted avg']['f1-score'] , result_bagging_30_before_data[sco]['weighted avg']['f1-score'], result_boosting_30_before_data[sco]['weighted avg']['f1-score']]
  accuracy_list_30_before_data[(sco+1)] = [result_max_voting_30_before_data[sco]['accuracy'] , result_bagging_30_before_data[sco]['accuracy'], result_boosting_30_before_data[sco]['accuracy']]
  precision_list_30_after_data[(sco+1)] = [result_max_voting_30_after_tuning[sco]['weighted avg']['precision'] , result_bagging_30_after_data[sco]['weighted avg']['precision'], result_boosting_30_after_data[sco]['weighted avg']['precision']]
  recall_list_30_after_data[(sco+1)] = [result_max_voting_30_after_tuning[sco]['weighted avg']['recall'] , result_bagging_30_after_data[sco]['weighted avg']['recall'], result_boosting_30_after_data[sco]['weighted avg']['recall']]
  f1_list_30_after_data[(sco+1)] = [result_max_voting_30_after_tuning[sco]['weighted avg']['f1-score'] , result_bagging_30_after_data[sco]['weighted avg']['f1-score'], result_boosting_30_after_data[sco]['weighted avg']['f1-score']]
  accuracy_list_30_after_data[(sco+1)] = [result_max_voting_30_after_tuning[sco]['accuracy'] , result_bagging_30_after_data[sco]['accuracy'], result_boosting_30_after_data[sco]['accuracy']]

In [53]:
print("")
print("")
print("{:<5} {:<0}".format("","Accuracy K-Fold 30% Data Before Tuning"))
print ("{:<8} {:<10} {:<15} {:<10}".format('Fold','MAX VOTING','BAGGING','BOOSTING'))
for k, v in accuracy_list_30_before_data.items():
    the_max_voting, the_bagging, the_boosting  = v
    print ("{:<8} {:<15} {:<10} {:<10}".format(k, format(the_max_voting,".3f"), format(the_bagging,".3f"), format(the_boosting,".3f") ))
print("")
print("")

print("")
print("")
print("{:<5} {:<0}".format("","Precision K-Fold 30% Data Before Tuning"))
print ("{:<8} {:<10} {:<15} {:<10}".format('Fold','MAX VOTING','BAGGING','BOOSTING'))
for k, v in precision_list_30_before_data.items():
    the_max_voting, the_bagging, the_boosting  = v
    print ("{:<8} {:<15} {:<10} {:<10}".format(k, format(the_max_voting,".3f"), format(the_bagging,".3f"), format(the_boosting,".3f") ))
print("")
print("")

print("")
print("")
print("{:<5} {:<0}".format("","Recall K-Fold 30% Data Before Tuning"))
print ("{:<8} {:<10} {:<15} {:<10}".format('Fold','MAX VOTING','BAGGING','BOOSTING'))
for k, v in recall_list_30_before_data.items():
    the_max_voting, the_bagging, the_boosting  = v
    print ("{:<8} {:<15} {:<10} {:<10}".format(k, format(the_max_voting,".3f"), format(the_bagging,".3f"), format(the_boosting,".3f") ))
print("")
print("")

print("")
print("")
print("{:<5} {:<0}".format("","F1-Score K-Fold 30% Data Before Tuning"))
print ("{:<8} {:<10} {:<15} {:<10}".format('Fold','MAX VOTING','BAGGING','BOOSTING'))
for k, v in f1_list_30_before_data.items():
    the_max_voting, the_bagging, the_boosting  = v
    print ("{:<8} {:<15} {:<10} {:<10}".format(k, format(the_max_voting,".3f"), format(the_bagging,".3f"), format(the_boosting,".3f") ))
print("")
print("")

print("")
print("")
print("{:<5} {:<0}".format("","Accuracy K-Fold 30% After Tuning"))
print ("{:<8} {:<10} {:<15} {:<10}".format('Fold','MAX VOTING','BAGGING','BOOSTING'))
for k, v in accuracy_list_30_after_data.items():
    the_max_voting, the_bagging, the_boosting  = v
    print ("{:<8} {:<15} {:<10} {:<10}".format(k, format(the_max_voting,".3f"), format(the_bagging,".3f"), format(the_boosting,".3f") ))
print("")
print("")

print("")
print("")
print("{:<5} {:<0}".format("","Precision K-Fold 30% Data After Tuning"))
print ("{:<8} {:<10} {:<15} {:<10}".format('Fold','MAX VOTING','BAGGING','BOOSTING'))
for k, v in precision_list_30_after_data.items():
    the_max_voting, the_bagging, the_boosting  = v
    print ("{:<8} {:<15} {:<10} {:<10}".format(k, format(the_max_voting,".3f"), format(the_bagging,".3f"), format(the_boosting,".3f") ))
print("")
print("")

print("")
print("")
print("{:<5} {:<0}".format("","Recall K-Fold 30% Data After Tuning"))
print ("{:<8} {:<10} {:<15} {:<10}".format('Fold','MAX VOTING','BAGGING','BOOSTING'))
for k, v in recall_list_30_after_data.items():
    the_max_voting, the_bagging, the_boosting  = v
    print ("{:<8} {:<15} {:<10} {:<10}".format(k, format(the_max_voting,".3f"), format(the_bagging,".3f"), format(the_boosting,".3f") ))
print("")
print("")

print("")
print("")
print("{:<5} {:<0}".format("","F1-Score K-Fold 30% Data After Tuning"))
print ("{:<8} {:<10} {:<15} {:<10}".format('Fold','MAX VOTING','BAGGING','BOOSTING'))
for k, v in f1_list_30_after_data.items():
    the_max_voting, the_bagging, the_boosting  = v
    print ("{:<8} {:<15} {:<10} {:<10}".format(k, format(the_max_voting,".3f"), format(the_bagging,".3f"), format(the_boosting,".3f") ))
print("")
print("")



      Accuracy K-Fold 30% Data Before Tuning
Fold     MAX VOTING BAGGING         BOOSTING  
1        0.936           0.957      0.701     
2        0.934           0.956      0.701     
3        0.935           0.956      0.701     
4        0.940           0.957      0.701     
5        0.935           0.956      0.702     




      Precision K-Fold 30% Data Before Tuning
Fold     MAX VOTING BAGGING         BOOSTING  
1        0.940           0.958      0.807     
2        0.937           0.956      0.806     
3        0.939           0.957      0.807     
4        0.942           0.957      0.806     
5        0.939           0.956      0.805     




      Recall K-Fold 30% Data Before Tuning
Fold     MAX VOTING BAGGING         BOOSTING  
1        0.936           0.957      0.701     
2        0.934           0.956      0.701     
3        0.935           0.956      0.701     
4        0.940           0.957      0.701     
5        0.935           0.956      0.702     




      